In [6]:
!pip -q install clickhouse-connect==0.6.11 transformers tokenizers torch torchvision sentencepiece protobuf

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
import clickhouse_connect
import pandas as pd
from transformers import pipeline

/usr/local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
client = clickhouse_connect.get_client(
    host="clickhouse",
    port=8123,
    username="clickhouse-user",
    password="secret"
)

In [3]:
query = """CREATE TABLE IF NOT EXISTS messages
(
    id                String,
    type              String,
    message           String,
    timestamp         Int64,
    datetime          String,
    elapsedTime       String,
    amountValue       Float32,
    amountString      String,
    currency          String,
    author_name       String,
    author_channelId  String,
    author_channelUrl String,
    video_id          String,
    stream_name       String
) ENGINE = MergeTree()
ORDER BY (timestamp);
"""

In [4]:
client.command(query)

In [7]:
data = {
    'id': 'ChwKGkNJcjhwWmlma29FREZiM0R3Z1FkT3VjTC1R', 
    'type': 'textMessage', 
    'message': 'lastima. Pinta de pokemones tienen jajaj', 
    'timestamp': 1693874754555, 
    'datetime': '2023-09-05 00:45:54', 
    'elapsedTime': '', 
    'amountValue': 0.0, 
    'amountString': '', 
    'currency': '', 
    'author_name': 'FerClonazepum!', 
    'author_channelId': 'UCIguRa3t5CYgm1ai_OuZq8A', 
    'author_channelUrl': 'http://www.youtube.com/channel/UCIguRa3t5CYgm1ai_OuZq8A'
}

In [28]:
data.keys()

dict_keys(['id', 'type', 'message', 'timestamp', 'datetime', 'elapsedTime', 'amountValue', 'amountString', 'currency', 'author_name', 'author_channelId', 'author_channelUrl'])

In [9]:
to_insert = data.values()
to_insert

dict_values(['ChwKGkNJcjhwWmlma29FREZiM0R3Z1FkT3VjTC1R', 'textMessage', 'lastima. Pinta de pokemones tienen jajaj', 1693874754555, '2023-09-05 00:45:54', '', 0.0, '', '', 'FerClonazepum!', 'UCIguRa3t5CYgm1ai_OuZq8A', 'http://www.youtube.com/channel/UCIguRa3t5CYgm1ai_OuZq8A'])

In [8]:
result = client.query("DESCRIBE TABLE messages")
result.result_rows

[('id', 'String', '', '', '', '', ''),
 ('type', 'String', '', '', '', '', ''),
 ('message', 'String', '', '', '', '', ''),
 ('timestamp', 'Int64', '', '', '', '', ''),
 ('datetime', 'String', '', '', '', '', ''),
 ('elapsedTime', 'String', '', '', '', '', ''),
 ('amountValue', 'Float32', '', '', '', '', ''),
 ('amountString', 'String', '', '', '', '', ''),
 ('currency', 'String', '', '', '', '', ''),
 ('author_name', 'String', '', '', '', '', ''),
 ('author_channelId', 'String', '', '', '', '', ''),
 ('author_channelUrl', 'String', '', '', '', '', ''),
 ('video_id', 'String', '', '', '', '', ''),
 ('stream_name', 'String', '', '', '', '', '')]

In [30]:
client.insert("messages", [[x for x in data.values()]], column_names=[x for x in data.keys()])

In [49]:
result = client.query("SELECT * FROM messages")

In [20]:
def make_df(result): 
    acc_columns = {x: [] for x in data.keys()}
    for x in result.result_rows:
        for curr_col, key in zip(x, data.keys()):
            acc_columns[key].append(curr_col)
            
    return pd.DataFrame(acc_columns)

In [21]:
make_df(result)

,id,type,message,timestamp,datetime,elapsedTime,amountValue,amountString,currency,author_name,author_channelId,author_channelUrl
0,ChwKGkNPM3d5LWVra29FREZaN0R3Z1FkanhVTk1R,textMessage,NADIE ES PATRIOTA....TODOS SON FANATICOS DE UN...,1693876263022,2023-09-05 01:11:03,,0.0,,,Lucho Berni,UCsf1cUShQHWu5SWhAttshxQ,http://www.youtube.com/channel/UCsf1cUShQHWu5S...
1,ChwKGkNOYXluS3lra29FREZUN1B3Z1FkcE4wTXl3,textMessage,liberad,1693876138518,2023-09-05 01:08:58,,0.0,,,nhnths,UCO5-Cr7nrd_HdnbXzSIJ2tg,http://www.youtube.com/channel/UCO5-Cr7nrd_Hdn...
2,ChwKGkNPX0pxYXlra29FREZhN0F3Z1Fkek0wTC1R,textMessage,AHHHHHHH QUE OFENDIDOS LOS PERIODISTAS MENTIRO...,1693876138726,2023-09-05 01:08:58,,0.0,,,Emiliano Rodriguez Tv,UC6FVqQ3URo1TAd1KpkS9bQw,http://www.youtube.com/channel/UC6FVqQ3URo1TAd...
3,ChwKGkNQVC05Nnlra29FREZUN1B3Z1FkcE4wTXl3,textMessage,Venimos a flotar:woman_fairy: y este nos estre...,1693876140011,2023-09-05 01:09:00,,0.0,,,ღ Mariah Amorosa ღ,UCVPFCFX8-i4g5Ls2M8kruwQ,http://www.youtube.com/channel/UCVPFCFX8-i4g5L...
4,ChwKGkNMNkRpYTJra29FREZYWEJ3Z1FkTGljT1l3,textMessage,Graciela a quien cancelaron si en C5N tienen l...,1693876140296,2023-09-05 01:09:00,,0.0,,,Fer,UCTptDAjBd3__28GLEIcvwJg,http://www.youtube.com/channel/UCTptDAjBd3__28...
...,...,...,...,...,...,...,...,...,...,...,...,...
68,ChwKGkNNMnNrT1Nra29FREZaN0R3Z1FkanhVTk1R,textMessage,yahoo q malo sos,1693876255752,2023-09-05 01:10:55,,0.0,,,Future,UCENzl7FgROKiRSLKyiQakSw,http://www.youtube.com/channel/UCENzl7FgROKiRS...
69,ChwKGkNNVEJ2T1Nra29FREZmeUY1UWNkRjZJTWV3,textMessage,Pelado sos re garca,1693876256486,2023-09-05 01:10:56,,0.0,,,Dakar,UCgPzirzz2eyArySq-brYBkA,http://www.youtube.com/channel/UCgPzirzz2eyAry...
70,ChwKGkNJNzR2ZVNra29FREZYN0N3Z1FkTzQ0TUt3,textMessage,lo ibertad,1693876256508,2023-09-05 01:10:56,,0.0,,,nhnths,UCO5-Cr7nrd_HdnbXzSIJ2tg,http://www.youtube.com/channel/UCO5-Cr7nrd_Hdn...
71,ChwKGkNKX3o4LVNra29FREZYX0R3Z1FkVWt3SkxB,textMessage,VIEJO MEADOOOOO,1693876257396,2023-09-05 01:10:57,,0.0,,,Joaquín Carrizo,UCxankCsqvqqH7jSHllFCRag,http://www.youtube.com/channel/UCxankCsqvqqH7j...


In [23]:
result = client.query("SELECT * FROM messages WHERE message LIKE '%milei%'")

In [24]:
make_df(result)

,id,type,message,timestamp,datetime,elapsedTime,amountValue,amountString,currency,author_name,author_channelId,author_channelUrl
0,ChwKGkNMU0EySy1ra29FREZhN0F3Z1Fkek0wTC1R,textMessage,milei es casta vagocracia,1693876145783,2023-09-05 01:09:05,,0.0,,,Gabriel Vera,UCWboC7ToDXQzEUsMiXgHaKA,http://www.youtube.com/channel/UCWboC7ToDXQzEU...
1,ChwKGkNLT0UwY0tra29FREZhVEN3Z1FkRTNVQW1R,textMessage,hoy se vió la derrota de milei:face-fuchsia-to...,1693876185512,2023-09-05 01:09:45,,0.0,,,Gabriel Vera,UCWboC7ToDXQzEUsMiXgHaKA,http://www.youtube.com/channel/UCWboC7ToDXQzEU...
2,ChwKGkNQakJ3NU9sa29FREZmYkJ3Z1FkZFd3QmpR,textMessage,milei,1693876355162,2023-09-05 01:12:35,,0.0,,,nhnths,UCO5-Cr7nrd_HdnbXzSIJ2tg,http://www.youtube.com/channel/UCO5-Cr7nrd_Hdn...
3,ChwKGkNPZUR5NmVsa29FREZiQWxyUVlkaFowRjln,textMessage,milei,1693876397228,2023-09-05 01:13:17,,0.0,,,nhnths,UCO5-Cr7nrd_HdnbXzSIJ2tg,http://www.youtube.com/channel/UCO5-Cr7nrd_Hdn...
4,ChwKGkNJbkQtN0tsa29FREZYX0R3Z1FkVWt3SkxB,textMessage,milei,1693876421091,2023-09-05 01:13:41,,0.0,,,nhnths,UCO5-Cr7nrd_HdnbXzSIJ2tg,http://www.youtube.com/channel/UCO5-Cr7nrd_Hdn...
5,ChwKGkNQbmJ2TS1sa29FREZhVEN3Z1FkRTNVQW1R,textMessage,milei:face-fuchsia-tongue-out: seguramente con...,1693876480883,2023-09-05 01:14:40,,0.0,,,JohnR777,UCgrDQ_tRZbE2PptMBvu1cGw,http://www.youtube.com/channel/UCgrDQ_tRZbE2Pp...
6,ChwKGkNMSGV4ZU9sa29FREZYN0N3Z1FkTzQ0TUt3,textMessage,Esos q dicen MIlei milei,1693876522970,2023-09-05 01:15:22,,0.0,,,Future,UCENzl7FgROKiRSLKyiQakSw,http://www.youtube.com/channel/UCENzl7FgROKiRS...
7,ChwKGkNMMkwxX3Fsa29FREZYWEJ3Z1FkTGljT1l3,textMessage,milei presidente yaaaaaa,1693876571491,2023-09-05 01:16:11,,0.0,,,Miguel Angel Cejas,UCqZ_We4GretHJIYV14Wf_vQ,http://www.youtube.com/channel/UCqZ_We4GretHJI...


In [ ]:
# SELECT message_content, author_id
# FROM messages
# GROUP BY message_content, author_id
# HAVING COUNT(*) > 1;

ored = "|".join(jxc_words)
result = client.query(f"SELECT * FROM messages WHERE LOWER(message) LIKE '{ored}'")
make_df(result)

In [ ]:
jxc_words = ["pato", "patricia", "bullrich", "jxc", "macri", "larreta", "pelado", "jorge", "mauricio"]
lla_words = ["libertad", "milei", "piparo", "villarruel", "javier", "leon"]
k_words = ["cristina", "kirchner", "fernandez", "massa", "sergio", "tomas", "kicillof", "alberto"]

In [ ]:
df = make_df(result)

In [57]:
columns = [x for x in df.columns]
jxc_acc = {x: [] for x in columns}
lla_acc = {x: [] for x in columns}
k_acc = {x: [] for x in columns}

for row in df.iterrows():
    message = row[1]["message"]
    for word in jxc_words:
        if word in message:
            for col in columns:
                jxc_acc[col].append(row[1][col])
            break

    for word in lla_words:
        if word in message:
            for col in columns:
                lla_acc[col].append(row[1][col])
            break
            
    for word in k_words:
        if word in message:
            for col in columns:
                k_acc[col].append(row[1][col])
            break
    
jxc_df = pd.DataFrame(jxc_acc)
lla_df = pd.DataFrame(lla_acc)
k_df = pd.DataFrame(k_acc)

In [63]:
jxc_df

,id,type,message,timestamp,datetime,elapsedTime,amountValue,amountString,currency,author_name,author_channelId,author_channelUrl
0,ChwKGkNLZURuY1Nra29FREZmWE1GZ2tkd1M4THJB,textMessage,Hacete cargo pelado,1693876188867,2023-09-05 01:09:48,,0.0,,,Dakar,UCgPzirzz2eyArySq-brYBkA,http://www.youtube.com/channel/UCgPzirzz2eyAry...
1,ChwKGkNQVDdsb3Fsa29FREZlR1Y1UWNkanlvTGt3,textMessage,que aburrido erl pelado este,1693876335566,2023-09-05 01:12:15,,0.0,,,Dakar,UCgPzirzz2eyArySq-brYBkA,http://www.youtube.com/channel/UCgPzirzz2eyAry...
2,ChwKGkNOTHItcjJua29FREZiM0R3Z1FkT3VjTC1R,textMessage,patricia no...,1693876981027,2023-09-05 01:23:01,,0.0,,,Nezuko,UC2iTys0FjSnt-tkdIpGgE9Q,http://www.youtube.com/channel/UC2iTys0FjSnt-t...


## Sentiment Analysis

In [7]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [8]:
pipe("Hola como estas?")

[{'label': 'neutral', 'score': 0.7640047073364258}]

## To Analyze

- Does it change with times?
- Does it change between channels?
- Sentiment between parties
- Amount of messages per candidate
- Clustering messages -> main bad things per candidate and main good things per candidate
- Word frequencies? -> Per time?
- Amount of messages per time?